In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：AutoML培训图像分类模型进行在线预测

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_image_classification_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 运行在Colab中
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_image_classification_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_image_classification_online.ipynb">
      在Google Cloud Notebooks中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex SDK创建图像分类模型并使用Google Cloud的AutoML模型进行在线预测。您可以查看[AutoML用户文档](https://cloud.google.com/vertex-ai/docs/start/automl-users)获取更多信息。

### 数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[花卉数据集](https://www.tensorflow.org/datasets/catalog/tf_flowers)。您在本教程中将使用的数据集版本存储在一个公共云存储桶中。训练好的模型可以预测一张图片属于五种花中的哪一种：雏菊、蒲公英、玫瑰、向日葵或郁金香。

### 目标

在本教程中，您将使用Vertex SDK从Python脚本创建一个AutoML图像分类模型，并部署用于在线预测。您也可以使用`gcloud`命令行工具或通过Cloud Console在线创建和部署模型。

执行的步骤包括：

- 创建一个Vertex`数据集`资源。
- 训练模型。
- 查看模型评估。
- 部署`模型`资源到一个用于提供服务的`终端点`资源。
- 进行预测。
- 解除部署`模型`。

成本

本教程使用了 Google Cloud 的可计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI
价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage
价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/)
基于您的预期使用情况生成成本估算。

### **设置本地开发环境**

如果您使用的是Colab或Google云笔记本，您的环境已经满足运行此笔记本的所有要求。您可以跳过这一步。

否则，请确保您的环境满足此笔记本的要求。您需要以下内容：

- 云存储SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了一组简化的说明：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，请在终端窗口中运行`pip3 install jupyter`命令。

5. 要启动Jupyter，请在终端窗口中运行`jupyter notebook`命令。

6. 在Jupyter Notebook仪表板中打开此笔记本。

## 安装

安装最新版本的Python用的Vertex SDK。

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade tensorflow $USER_FLAG

### 重新启动内核

安装了额外的包之后，您需要重新启动笔记本内核，以便它能够找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU 运行时

本教程不需要使用 GPU 运行时。

### 设置您的 Google Cloud 项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用额度，可用于计算和存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI APIs，Compute Engine APIs和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您是在本地运行此笔记本，您需要安装 [Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保 Cloud SDK 在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 运行以 `!` 为前缀的行作为shell命令，并会对以 `$` 为前缀的Python变量进行插值。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改`REGION`变量，该变量用于本笔记本的其余操作。以下是Vertex AI支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您不能使用多区域存储桶进行Vertex AI训练。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在参加一个现场教程会话，您可能正在使用一个共享的测试帐户或项目。为了避免在创建的资源上发生名称冲突，您为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云账户

**如果您正在使用谷歌云笔记本**，您的环境已经得到验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按提示执行以通过oAuth验证您的账户。

**否则**，请按照以下步骤操作：

在云控制台中，前往[创建服务账户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账户**。

在**服务账户名称**字段中输入一个名称，然后点击**创建**。

在**授予该服务账户对项目的访问权限**部分，点击角色下拉列表。在筛选框中输入"Vertex"，然后选择**Vertex管理员**。在筛选框中输入"Storage Object Admin"，然后选择**存储对象管理员**。

点击创建。包含您密钥的JSON文件将下载到您的本地环境中。

在下面的单元格中，将您服务账户密钥的路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

创建一个云存储桶

**无论您所使用的笔记本环境如何，都需要执行以下步骤。**

当您初始化用 Python 编写的 Vertex SDK 时，需要指定一个云存储暂存桶。这个暂存桶是存储与数据集和模型资源相关的所有数据，跨会话保持不变。

请在下方设置您的云存储桶的名称。存储桶的名称必须在所有 Google Cloud 项目中唯一，包括您所属组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶不存在时才能运行以下单元格来创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。

### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

## 初始化Python的Vertex SDK

为您的项目和相应的存储桶初始化Python的Vertex SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

# 教程

现在你已经准备好开始创建自己的AutoML图像分类模型了。

#### 云存储培训数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
)

快速查看您的数据

本教程使用存储在公共云存储桶中的花卉数据集的一个版本，使用CSV索引文件。

首先快速查看数据。通过在CSV索引文件(`wc -l`)中计算行数来计算示例的数量，然后查看前几行。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 创建数据集

接下来，使用`ImageDataset`类的`create`方法创建`Dataset`资源，该方法接受以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：要将数据项目导入`Dataset`资源的一个或多个数据集索引文件列表。
- `import_schema_uri`：数据项目的数据标签架构。

此操作可能需要几分钟的时间。

In [ ]:
dataset = aip.ImageDataset.create(
    display_name="Flowers" + "_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aip.schema.dataset.ioformat.image.single_label_classification,
)

print(dataset.resource_name)

### 创建并运行训练流程

要训练一个AutoML模型，需要执行两个步骤：1）创建一个训练流程，2）运行这个流程。

#### 创建训练流程

使用`AutoMLImageTrainingJob`类创建一个AutoML训练流程，需要以下参数：

- `display_name`：`TrainingJob`资源的可读名称。
- `prediction_type`：为模型训练指定任务类型。
  - `classification`：图像分类模型。
  - `object_detection`：图像目标检测模型。
- `multi_label`：如果是分类任务，指定是单标签（`False`）还是多标签（`True`）。
- `model_type`：部署模型的类型。
  - `CLOUD`：在Google Cloud上部署。
  - `CLOUD_HIGH_ACCURACY_1`：为在Google Cloud上部署时优化准确性而非延迟。
  - `CLOUD_LOW_LATENCY_1`：为在Google Cloud上部署时优化延迟而非准确性。
  - `MOBILE_TF_VERSATILE_1`：在边缘设备上部署。
  - `MOBILE_TF_HIGH_ACCURACY_1`：为在边缘设备上部署时优化准确性而非延迟。
  - `MOBILE_TF_LOW_LATENCY_1`：为在边缘设备上部署时优化延迟而非准确性。
- `base_model`：（可选）从现有的`Model`资源进行迁移学习，仅对图像分类有效。

实例化的对象是训练作业的有向无环图(DAG)。

In [ ]:
dag = aip.AutoMLImageTrainingJob(
    display_name="flowers_" + TIMESTAMP,
    prediction_type="classification",
    multi_label=False,
    model_type="CLOUD",
    base_model=None,
)

print(dag)

#### 运行训练管道

接下来，通过调用方法`run`，使用以下参数来运行DAG以启动训练作业：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：训练模型的可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留置数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `budget_milli_node_hours`：（可选）以千小时为单位指定的最大训练时间。
- `disable_early_stopping`：如果为`True`，则当服务认为无法进一步改善模型目标测量时，训练可能会在使用整个预算之前完成。

完成`run`方法后会返回`Model`资源。

训练管道的执行将持续最多20分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="flowers_" + TIMESTAMP,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
)

## 查看模型评估分数
在您的模型训练完成后，您可以查看其评估分数。

首先，您需要获取新模型的引用。与数据集一样，您可以使用部署模型时创建的模型变量的引用，或者列出项目中的所有模型。

In [ ]:
# Get model resource ID
models = aip.Model.list(filter="display_name=flowers_" + TIMESTAMP)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

部署模型

接下来，部署您的模型用于在线预测。要部署模型，您需要调用`deploy`方法。

In [ ]:
endpoint = model.deploy()

发送一个在线预测请求

向您部署的模型发送一个在线预测。

获取测试项

您将从数据集中选择一个任意的示例作为测试项。不必担心该示例可能已在训练模型中使用过 - 我们只是想展示如何进行预测。

In [ ]:
test_item = !gsutil cat $IMPORT_FILE | head -n1
if len(str(test_item[0]).split(",")) == 3:
    _, test_item, test_label = str(test_item[0]).split(",")
else:
    test_item, test_label = str(test_item[0]).split(",")

print(test_item, test_label)

### 进行预测

现在，您的`Model`资源已部署到一个`Endpoint`资源，您可以通过向Endpoint资源发送预测请求来进行在线预测。

#### 请求

在这个示例中，由于您的测试项目位于云存储桶中，您可以使用`tf.io.gfile.Gfile()`打开并读取图像内容。为了将测试数据传递给预测服务，您需要将字节编码为base64 -- 这样可以使内容在通过网络传输二进制数据时免受修改。

每个实例的格式为：

    { 'content': { 'b64': base64编码的字节 } }

由于`predict()`方法可以接受多个项目（实例），请将您的单个测试项目作为一个测试项目列表发送。

#### 回应

从`predict()`调用中得到的回应是一个Python字典，包含以下条目：

- `ids`：每个预测请求的内部分配唯一标识符。
- `displayNames`：每个类标签的类名。
- `confidences`：每个类标签的预测置信度，介于0和1之间。
- `deployed_model_id`：执行预测的Vertex AI标识符，它指代部署的Model资源。 

In [ ]:
import base64

import tensorflow as tf

with tf.io.gfile.GFile(test_item, "rb") as f:
    content = f.read()

# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{"content": base64.b64encode(content).decode("utf-8")}]

prediction = endpoint.predict(instances=instances)

print(prediction)

解除部署模型

当您完成预测时，您会从“端点”资源中解除部署模型。这将取消所有计算资源的分配，并结束为部署模型而进行的计费。

In [ ]:
endpoint.undeploy_all()

清理工作

要清理本项目中使用的所有谷歌云资源，您可以删除用于本教程的[谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 管道
- 模型
- 终端
- AutoML 训练作业
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME